In [41]:
import pandas as pd
import numpy as np
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import subprocess
import json
from os import listdir
from os.path import isfile, join
import datetime
import time

In [42]:
start = time.time()

files=pd.DataFrame(columns=['folder','path_audio'])

dir_audios="../Data/Audio"
dir_audios= dir_audios+"/" if dir_audios[-1]!="/" else dir_audios
folders_audio = [dir_audios+f for f in listdir(dir_audios)]

for folders in folders_audio:
    file_audio = [folders+"/"+f for f in listdir(folders)]
    file_audio = pd.DataFrame(file_audio,columns=['path_audio'])
    file_audio['folder'] = folders.split("/")[-1]

    files = files.append(file_audio)

In [43]:
files['file']=files['path_audio'].apply(lambda x: x.split("/")[-1])
aux = pd.Series(files['file'].apply(lambda x: x.replace(".wav","").split("-")[-6:]))
aux = aux.apply(lambda x: [int(a) if len(a)<=4 else int(a[-4:]) for a in x])
files['date'] = aux.apply(lambda x: datetime.datetime(x[0], x[1], x[2], hour=x[3], minute=x[4], second=x[5]))

files.reset_index(drop=True, inplace=True)
files

,folder,path_audio,file,date
0,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23
1,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49
2,TN,../Data/Audio/TN/TN2020-08-11-19-01-34.wav,TN2020-08-11-19-01-34.wav,2020-08-11 19:01:34
3,TN,../Data/Audio/TN/TN2020-08-14-18-59-25.wav,TN2020-08-14-18-59-25.wav,2020-08-14 18:59:25


In [44]:
prueba = pd.DataFrame([34],columns=["time"])


In [45]:
folder_model="../Data/Modelos/audio_model_spanish_vosk"
model = Model(folder_model)

registros = pd.DataFrame(columns=['path_audio','result','text'])
for file in files['path_audio']:
    wf = wave.open(file, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())    

    while True:
        data = wf.readframes(1000000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            try:
                new_input = json.loads(rec.Result())
                new_input = pd.DataFrame(new_input)
                new_input['path_audio']=file
                registros = registros.append(new_input)
            except ValueError:
                 continue
        else:
            rec.PartialResult()


In [46]:
registros.reset_index(drop=True,inplace=True)
registros_2 = registros.join(pd.DataFrame(x for x in registros['result']))
registros_2.drop(columns='result',inplace=True)
registros_2.head()

,path_audio,text,conf,end,start,word
0,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,el doctor adolfo ruiz stein que había dicho qu...,1.000000,0.900000,0.720000,el
1,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,el doctor adolfo ruiz stein que había dicho qu...,1.000000,1.230000,0.900000,doctor
2,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,el doctor adolfo ruiz stein que había dicho qu...,1.000000,1.650000,1.230000,adolfo
3,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,el doctor adolfo ruiz stein que había dicho qu...,0.999722,2.159989,1.680000,ruiz
4,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,el doctor adolfo ruiz stein que había dicho qu...,0.999985,2.400000,2.159989,stein


In [48]:
MinTime = pd.DataFrame(registros_2.groupby(['path_audio','text'])['start'].min())
MaxTime = pd.DataFrame(registros_2.groupby(['path_audio','text'])['end'].max())

registros_3=registros_2[['path_audio','text']]
registros_3 = registros_3.merge(MinTime,on=['path_audio','text'])
registros_3 = registros_3.merge(MaxTime,on=['path_audio','text'])

registros_3.drop_duplicates(inplace=True)
registros_3.reset_index(drop=True,inplace=True)


registros_3.head()

,path_audio,text,start,end
0,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,el doctor adolfo ruiz stein que había dicho qu...,0.72,44.520000
1,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,muy mal o dejaron sin vacunas a la argentina s...,45.39,90.300000
2,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,demostrar como demostró siempre le importan tr...,90.75,135.330000
3,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,deberían pedir a rubita y que no hable más es ...,135.81,180.990000
4,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,ocho mil diez mil y la ciudad siempre desea do...,181.02,225.359209


In [50]:
data = files.merge(registros_3, on='path_audio')

data['start'] = data['start'] .apply(lambda x: datetime.timedelta(seconds=int(x)))
data['start'] = data['start'] + data['date']
data['end'] = data['end'] .apply(lambda x: datetime.timedelta(seconds=int(x)))
data['end'] = data['end'] + data['date']

print(data.shape)
data.head()

(528, 7)


,folder,path_audio,file,date,text,start,end
0,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23,el doctor adolfo ruiz stein que había dicho qu...,2020-08-11 17:59:23,2020-08-11 18:00:07
1,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23,muy mal o dejaron sin vacunas a la argentina s...,2020-08-11 18:00:08,2020-08-11 18:00:53
2,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23,demostrar como demostró siempre le importan tr...,2020-08-11 18:00:53,2020-08-11 18:01:38
3,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23,deberían pedir a rubita y que no hable más es ...,2020-08-11 18:01:38,2020-08-11 18:02:23
4,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23,ocho mil diez mil y la ciudad siempre desea do...,2020-08-11 18:02:24,2020-08-11 18:03:08


In [51]:
data_text_complete  = pd.DataFrame(columns=['path_audio','text_complete'])
for file in data['path_audio'].unique():
    text = data.loc[data['path_audio'] == file,'text'].unique()
    text_complete = ','.join(text)
    new_input=pd.DataFrame([text_complete], columns=['text_complete'])
    new_input['path_audio'] = file
    data_text_complete = data_text_complete.append(new_input)

data_text_complete.reset_index(drop=True,inplace=True)
data_text_complete=data[['folder','path_audio','file','date']].drop_duplicates().merge(data_text_complete, on = 'path_audio')

In [52]:
pd.set_option('display.max_colwidth',50) ##el segundo parametro indica la cantidad de caracteres a mostrar, con valor -1 muestra todo
data_text_complete

,folder,path_audio,file,date,text_complete
0,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23,el doctor adolfo ruiz stein que había dicho qu...
1,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,a hola hola muy buenas tardes bienvenidos al d...
2,TN,../Data/Audio/TN/TN2020-08-11-19-01-34.wav,TN2020-08-11-19-01-34.wav,2020-08-11 19:01:34,de ciento cincuenta mil casos corresponder al ...
3,TN,../Data/Audio/TN/TN2020-08-14-18-59-25.wav,TN2020-08-14-18-59-25.wav,2020-08-14 18:59:25,de los colores que se habían ido arrojó de las...


In [53]:
##Export datasets
data.to_csv("../Data procesada/data_audio.csv")
data_text_complete.to_csv("../Data procesada/data_audio_concatenada.csv")

In [54]:
end = time.time()
print("Tiempo de procesamiento:",round((end - start)/60),"minutos")

Tiempo de procesamiento: 41 minutos
